<a href="https://colab.research.google.com/github/Solostriker/Real-Time-Stock-Predictions/blob/main/Real_Time_Stock_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install intrinio-sdk

import intrinio_sdk as intrinio
import numpy as np
import pandas as pd
import time

intrinio.ApiClient().set_api_key('OjljOTM0MmQwMjEyMzM2MWY5YmYzNTM5ZjEwOTE1NTZh')

identifier = 'AAPL'
source = 'nasdaq_basic'

response = intrinio.SecurityApi().get_security_realtime_price(identifier, source=source)
print(response)

ApiException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'Date': 'Mon, 26 Feb 2024 13:11:23 GMT', 'Content-Type': 'application/json', 'Content-Length': '104', 'Connection': 'keep-alive', 'Vary': 'Origin,Accept-Encoding'})
HTTP response body: {"human":"No active subscription(s).","message":"An active subscription is required to view this data."}


In [ ]:
class QLearningTrader:

    def __init__(self, num_actions, num_features, learning_rate, discount_factor, exploration_prob):
        self.num_actions = num_actions
        self.num_features = num_features
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.exploration_prob = exploration_prob

        self.q_table = np.zeros((num_actions, num_features))

        self.current_state = None
        self.current_action = None

In [ ]:
def choose_action(self, state):

        if np.random.uniform(0, 1) < self.exploration_prob:
            return np.random.choice(self.num_actions)
        else:
            feature_index = np.argmax(state)
            return np.argmax(self.q_table[:, feature_index])

In [ ]:
def calculate_reward(action, current_close, next_close):

    if action == 0:
        return 1.0 if next_close > current_close else -1.0
    elif action == 1:
        return 1.0 if next_close < current_close else -1.0
    else:
        return 1.0 if next_close > current_close else -1.0 if next_close < current_close else 0.0

In [ ]:
def fetch_real_time_data(identifier):

    source = 'nasdaq_basic'
    response = intrinio.SecurityApi().get_security_realtime_price(identifier, source=source)

    return {
        'open': response.open_price,
        'high': response.high_price,
        'low': response.low_price,
        'close': response.last_price,
        'volume': response.last_size
    }

In [ ]:
def observe_real_time_data(self,identifier):
  real_time_data = fetch_real_time_data(identifier)

  self.current_state = np.array([real_time_data['open'], real_time_data['high'],real_time_data['low'], real_time_data['close'],real_time_data['volume']])

def observe_next_state(self,identifier):
  self.current_state = fetch_real_time_data(identifier)

In [ ]:
def take_action(self, action, reward):

  if self.current_action is not None:
    feature_index = np.argmax(self.current_state)
    current_q_value = self.q_table[self.current_action, feature_index]
    new_q_value = (1 - self.learning_rate) * current_q_value + self.learning_rate * (reward + self.discount_factor *np.max(self.q_table[:, feature_index]))
    self.q_table[self.current_action, feature_index] = new_q_value

  self.current_state = None
  self.current_action = action

In [ ]:
def calculate_profit_loss(initial_balance, suggested_action, current_close, next_close, quantity):

    if suggested_action == "Buy":
        return (next_close - current_close) * quantity
    elif suggested_action == "Sell":
        return (current_close - next_close) * quantity
    else:
        return 0.0

In [ ]:
def calculate_final_profit(identifier, initial_balance, quantity, num_iterations, learning_rate, discount_factor, exploration_prob):
    num_actions = 3
    num_features = 5
    q_trader = QLearningTrader(num_actions, num_features, learning_rate, discount_factor, exploration_prob)

    for i in range(num_iterations):
        q_trader.observe_real_time_data(identifier)

        action = q_trader.choose_action(q_trader.current_state)
        current_close = q_trader.current_state[3]

        time.sleep(1)

        q_trader.observe_next_state(identifier)
        next_close = q_trader.current_state['close']

        reward = calculate_reward(action, current_close, next_close)
        q_trader.take_action(action, reward)

    final_real_time_data = fetch_real_time_data(identifier)

    final_suggested_action = ["Buy", "Sell", "Hold"][np.argmax(q_trader.q_table[:, np.argmax(q_trader.current_state)])]

    final_profit = calculate_profit_loss(initial_balance, final_suggested_action, current_close, final_real_time_data['close'], quantity)

    print(f"Final Suggested Action: {final_suggested_action}, Final Profit: {final_profit}")

In [ ]:
class QLearningTrader:
    def __init__(self, num_actions, num_features, learning_rate, discount_factor, exploration_prob):
        self.num_actions = num_actions
        self.num_features = num_features
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.exploration_prob = exploration_prob

        self.q_table = np.zeros((num_actions, num_features))

        self.current_state = None
        self.current_action = None

    def choose_action(self, state):
        if np.random.uniform(0, 1) < self.exploration_prob:
            return np.random.choice(self.num_actions)
        else:
            feature_index = np.argmax(state)
            return np.argmax(self.q_table[:, feature_index])

    def take_action(self, action, reward):
        if self.current_action is not None:
            feature_index = np.argmax(self.current_state)
            current_q_value = self.q_table[self.current_action, feature_index]
            new_q_value = (1 - self.learning_rate) * current_q_value + \
                           self.learning_rate * (reward + self.discount_factor * np.max(self.q_table[:, feature_index]))
            self.q_table[self.current_action, feature_index] = new_q_value

        self.current_state = None
        self.current_action = action

    def observe_real_time_data(self, identifier):
        real_time_data = fetch_real_time_data(identifier)

        self.current_state = np.array([real_time_data['open'], real_time_data['high'],
                                       real_time_data['low'], real_time_data['close'],
                                       real_time_data['volume']])

    def observe_next_state(self, identifier):
        self.current_state = fetch_real_time_data(identifier)

def fetch_real_time_data(identifier):
    source = 'nasdaq_basic'
    response = intrinio.SecurityApi().get_security_realtime_price(identifier, source=source)

    return {
        'open': response.open_price,
        'high': response.high_price,
        'low': response.low_price,
        'close': response.last_price,
        'volume': response.last_size
    }

def calculate_reward(action, current_close, next_close):
    if action == 0:
        return 1.0 if next_close > current_close else -1.0
    elif action == 1:
        return 1.0 if next_close < current_close else -1.0
    else:
        return 1.0 if next_close > current_close else -1.0 if next_close < current_close else 0.0

def calculate_profit_loss(initial_balance, suggested_action, current_close, next_close, quantity):
    if suggested_action == "Buy":
        return (next_close - current_close) * quantity
    elif suggested_action == "Sell":
        return (current_close - next_close) * quantity
    else:
        return 0.0

def calculate_final_profit(identifier, initial_balance, quantity, num_iterations, learning_rate, discount_factor, exploration_prob):
    num_actions = 3
    num_features = 5
    q_trader = QLearningTrader(num_actions, num_features, learning_rate, discount_factor, exploration_prob)

    for i in range(num_iterations):
        q_trader.observe_real_time_data(identifier)

        action = q_trader.choose_action(q_trader.current_state)
        current_close = q_trader.current_state[3]

        time.sleep(1)

        q_trader.observe_next_state(identifier)
        next_close = q_trader.current_state['close']

        reward = calculate_reward(action, current_close, next_close)
        q_trader.take_action(action, reward)

    final_real_time_data = fetch_real_time_data(identifier)

    final_suggested_action = ["Buy", "Sell", "Hold"][np.argmax(q_trader.q_table[:, np.argmax(q_trader.current_state)])]

    final_profit = calculate_profit_loss(initial_balance, final_suggested_action, current_close, final_real_time_data['close'], quantity)

    print(f"Final Suggested Action: {final_suggested_action}, Final Profit: {final_profit}")
Now, let’s conduct the ultimate experiment to analyze the results:


security_identifier = 'AAPL'
calculate_final_profit(security_identifier, initial_balance = 100, quantity = 10, num_iterations = 180, learning_rate = 0.1, discount_factor = 0.9, exploration_prob = 0.2)